In [1]:
import pandas as pd

In [4]:
crashes = pd.read_csv("../data/dwh/CrashesFact.csv")
drivers = pd.read_csv("../data/dwh/DriversFact.csv")
roaddim = pd.read_csv("../data/dwh/RoadDim.csv")
vehicles = pd.read_csv("../data/dwh/VehicleDim.csv")

In [5]:
len(drivers)

172105

In [6]:
fact_merged = drivers.merge(crashes, on='ReportNumber')

In [7]:
len(fact_merged)

172105

In [8]:
fact_merged

,ReportNumber,VehicleID,DriverAtFault,InjurySeverity,DriverSubstanceAbuse,DriverDistractedBy,VehicleType,VehicleMovement,VehicleGoingDir,SpeedLimit,...,RoadAlignment,RoadCondition,RoadDivision,Latitude,Longitude,RoadKey,CrossStreetKey,NonMotoristTotal,NonMotoristInjury,NonMotoristFatal
0,MCP3170003V,4E492574-893B-4EB1-ADCA-53FDD633D6C4,True,NO APPARENT INJURY,NONE,LOOKED BUT DID NOT SEE,PASSENGER,MOVING CONSTANT SPEED,South,0,...,UNKNOWN,UNKNOWN,UNKNOWN,39.165005,-77.249310,UNKNOWN_UNKNOWN,UNKNOWN_UNKNOWN,0,0,0
1,MCP3170003V,904F2114-2B56-46D9-B105-356073FA1D8A,True,NO APPARENT INJURY,NONE,LOOKED BUT DID NOT SEE,BUS,MOVING CONSTANT SPEED,North,0,...,UNKNOWN,UNKNOWN,UNKNOWN,39.165005,-77.249310,UNKNOWN_UNKNOWN,UNKNOWN_UNKNOWN,0,0,0
2,MCP3254003K,6D16232C-4E1E-49A6-B3A2-7FDEF7E506F2,False,NO APPARENT INJURY,NONE,NOT DISTRACTED,PASSENGER,MOVING CONSTANT SPEED,West,35,...,STRAIGHT,NO DEFECTS,"TWO-WAY, DIVIDED, POSITIVE MEDIAN BARRIER",39.178776,-77.267190,GERMANTOWN_RD_Maryland,MIDDLEBROOK_RD_County,1,1,0
3,EJ7887003Q,82ED056D-33FA-44A5-AA2B-8DCE653FA03C,False,SUSPECTED MINOR INJURY,NONE,NOT DISTRACTED,PASSENGER,MOVING CONSTANT SPEED,South,35,...,STRAIGHT,NO DEFECTS,"TWO-WAY, DIVIDED, POSITIVE MEDIAN BARRIER",39.123574,-77.231769,GREAT_SENECA_HWY_Maryland,KENTLANDS_BLVD_Municipality,0,0,0
4,EJ7887003Q,04B387D7-FE5C-47B4-802A-7C8448274F4B,True,NO APPARENT INJURY,NONE,NOT DISTRACTED,PASSENGER,MOVING CONSTANT SPEED,South,35,...,STRAIGHT,NO DEFECTS,"TWO-WAY, DIVIDED, POSITIVE MEDIAN BARRIER",39.123574,-77.231769,GREAT_SENECA_HWY_Maryland,KENTLANDS_BLVD_Municipality,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172100,MCP1148000V,19FDF33C-F67F-4519-8543-DF976687A379,True,NO APPARENT INJURY,NONE,NOT DISTRACTED,TRUCK,PARKED,UNKNOWN,25,...,CURVE RIGHT,NO DEFECTS,"TWO-WAY, NOT DIVIDED",38.981540,-77.095787,WOODMONT_AVE_County,BETHESDA_AVE_County,1,1,0
172101,MCP20080083,6607F246-29CA-4C5E-A956-F7B3930387D6,True,NO APPARENT INJURY,NONE,NOT DISTRACTED,PASSENGER,MAKING LEFT TURN,West,40,...,STRAIGHT,NO DEFECTS,"TWO-WAY, DIVIDED, POSITIVE MEDIAN BARRIER",39.023230,-77.162160,DEMOCRACY_BLVD_County,SEVEN_LOCKS_RD_County,1,1,0
172102,MCP32710015,D907E819-FE51-488C-9315-17138E15D026,True,NO APPARENT INJURY,NONE,UNKNOWN,PASSENGER,PARKING,North,5,...,UNKNOWN,UNKNOWN,UNKNOWN,38.962411,-77.087952,UNKNOWN_UNKNOWN,UNKNOWN_UNKNOWN,0,0,0
172103,DD5649002Q,76A1A34D-B048-489D-BC22-461DD2BB7D35,True,SUSPECTED MINOR INJURY,ALCOHOL,NOT DISTRACTED,PASSENGER,MOVING CONSTANT SPEED,North,25,...,CURVE LEFT,NO DEFECTS,"TWO-WAY, DIVIDED, UNPROTECTED PAINTED MIN 4 FEET",39.074504,-77.111280,TWINBROOK_PKWY_Municipality,MCAULIFFE_DR_Municipality,0,0,0


In [9]:
fact_vehicles = fact_merged.merge(vehicles, on='VehicleKey')

In [10]:
len(fact_vehicles)

172105

In [15]:
all = fact_vehicles.merge(roaddim, left_on='RoadKey', right_on='RoadKey').merge(roaddim, left_on='CrossStreetKey', right_on='RoadKey')

In [16]:
all.columns

Index(['ReportNumber', 'VehicleID', 'DriverAtFault', 'InjurySeverity',
       'DriverSubstanceAbuse', 'DriverDistractedBy', 'VehicleType',
       'VehicleMovement', 'VehicleGoingDir', 'SpeedLimit', 'ParkedVehicle',
       'SubstanceAbuseContributed', 'VehicleKey', 'LocalCaseNumber',
       'AgencyName', 'ACRSReportType', 'Datetime', 'HitRun', 'LaneDirection',
       'LaneNumber', 'NumberOfLanes', 'RoadGrade', 'NonTraffic',
       'OffRoadIncident', 'AccidentAtFault', 'CollisionType',
       'SurfaceCondition', 'Light', 'TrafficControl', 'Junction',
       'IntersectionType', 'RoadAlignment', 'RoadCondition', 'RoadDivision',
       'Latitude', 'Longitude', 'RoadKey_x', 'CrossStreetKey',
       'NonMotoristTotal', 'NonMotoristInjury', 'NonMotoristFatal', 'Make',
       'Year', 'BaseModel', 'BodyClass', 'Cylinders', 'Displacement',
       'Transmission', 'Drivetrain', 'FuelType', 'CityMPG', 'HighwayMPG',
       'RoadName_x', 'RouteType_x', 'RoadName_y', 'RouteType_y', 'RoadKey_y'],
      

In [17]:
len(all)

172105